In [5]:
pip uninstall deepchem

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(f"Number of available CUDA devices: {torch.cuda.device_count()}")
print(f"Current CUDA device index: {torch.cuda.current_device()}")
print(f"Name of current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

Number of available CUDA devices: 1
Current CUDA device index: 0
Name of current CUDA device: NVIDIA GeForce GTX 1060 6GB


In [1]:
import pandas as pd 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import random
import deepchem as dc
from deepchem.models import MEGNetModel
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (d:\Projects\projects_python\deepchem_energy_pred_megnet\.venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
data = pd.read_csv('../megnet_test/data/data_all_new.csv')

molecules = []
targets = []
bad_mol = []

featurizer_func = dc.feat.RDKitConformerFeaturizer()

for i, row in tqdm(data.iterrows(), total=data.shape[0]):
    mol = Chem.Mol(Chem.MolFromXYZBlock(row['xyz']))
    try:
        rdDetermineBonds.DetermineBonds(mol)
        molecules.append(featurizer_func(mol)[0])
        targets.append(row['U_0'] / len(mol.GetAtoms()))
    except ValueError:
        bad_mol.append((i, mol))


  0%|          | 461/122381 [00:02<09:23, 216.26it/s]


KeyboardInterrupt: 

In [118]:
len(bad_mol)

11444

In [145]:
type(molecules[1033])

numpy.ndarray

In [3]:
mol_new = []
tar_new = []
for i in range(len(molecules)):
    if type(molecules[i]) != np.ndarray:
        mol_new.append(molecules[i])
        tar_new.append(targets[i])

In [4]:
mol1 = Chem.Mol(Chem.MolFromXYZBlock(data.iloc[1]['xyz']))
mol1
rdDetermineBonds.DetermineBonds(mol1)
featurizer_func = dc.feat.RDKitConformerFeaturizer()
featurizer_func(mol1)[0]

GraphData(node_features=[9, 9], edge_index=[2, 16], edge_features=[16, 3], global_features=[1, 1])

In [6]:
dataset = dc.data.NumpyDataset(X=mol_new, y=tar_new)

In [155]:
from deepchem.utils.fake_data_generator import FakeGraphGenerator as FGG
graphs = FGG(global_features=4, num_classes=10).sample(n_graphs=20)
graphs.X[0]

GraphData(node_features=[10, 5], edge_index=[2, 32], edge_features=[32, 3], global_features=[1, 4], num_classes=[1, 10])

In [10]:
import deepchem as dc
from deepchem.models import MEGNetModel
model = dc.models.MEGNetModel(n_node_features=8, n_edge_features=3, n_global_features=3, n_blocks=3, is_undirected=True, residual_connection=True, mode='regression')
model.fit(dataset.to())

22


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [97]:
graphs

<NumpyDataset X.shape: (20,), y.shape: (20,), w.shape: (20,), ids: [0 1 2 ... 17 18 19], task_names: [0]>

In [105]:
dataset.X[0][0].to_pyg_graph()

Data(x=[9, 30], edge_index=[2, 16])

In [10]:
def create_structure_dict(structures: list, targets: list):
    structure_dict = {}

    for structure in tqdm(zip(structures, targets), total=len(structures)):
        if structure[0].formula in structure_dict:
            structure_dict[structure[0].formula].append(structure)
        else:
            structure_dict[structure[0].formula] = [structure]
        
    return structure_dict

def train_test_split_for_structures(structure_dict: dict, test_size: float = 0.1):
    structures_train, structures_test = [], []
    targets_train, targets_test = [], []
    train_all, test_all = [], []

    for _, data in tqdm(structure_dict.items()):
        if len(data) < 2:
            train_all += data
        else:
            train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
            train_all += train_data
            test_all += test_data
    
    random.shuffle(train_all)
    random.shuffle(test_all)

    for t_data in train_all:
        structures_train.append(t_data[0])
        targets_train.append(t_data[1])
    
    for te_data in test_all:
        structures_test.append(te_data[0])
        targets_test.append(te_data[1])
    
    return structures_train, targets_train, structures_test, targets_test

In [11]:
structure_dict = create_structure_dict(molecules, targets)
molecules_train, targets_train, molecules_val_test, targets_val_test = train_test_split_for_structures(structure_dict)
structure_dict_val_test = create_structure_dict(molecules_val_test, targets_val_test)
molecules_val, targets_val, molecules_test, targets_test = train_test_split_for_structures(structure_dict_val_test, 0.5)

100%|██████████| 117/117 [00:00<00:00, 4184.33it/s]


In [ ]:
model = dc.models.MEGNetModel(n_node_features=5, n_edge_features=3, n_global_features=4, n_blocks=3, is_undirected=True, residual_connection=True, mode='classification', n_classes=10, batch_size=16)
training_loss = model.fit()